In [15]:
import re
import xml.dom.minidom

def xpath2xml(xpath, xmlns='', operation=None):
    #transforms xpath like string e.g. "/System/eps-items/epId-items/Ep-list/epId=1/nws-items/vni-items/Nw-list[]/vni=10444"
    #to xml-like sequence of elements tags: 
    # <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
    #   <eps-items>
    #       <epId-items>
    #           <Ep-list>
    #               <epId>1</epId>
    #               <nws-items>
    #                   <vni-items>
    #                       <Nw-list operation="remove">
    #                           <vni>10444</vni>
    #                       </Nw-list>
    #                   </vni-items>
    #               </nws-items>
    #           </Ep-list>
    #       </epId-items>
    #   </eps-items>
    # </System>
    #
    # it has optional parameter 'operation' that adds the string 'operation="value"' to the element marked with square brackets '[]'
    # RFC6241 https://tools.ietf.org/html/rfc6241#page-37
    #
    # there is a problem with parsing elements that contain the '/' separator within like this tDn element has:
    # /System/intf-items/svi-items/If-list/rtvrfMbr-items/tDn=/System/inst-items/Inst-list[name='{vrf_name}']
    # the same case if you want to configure a physical interface like Eth103/1/20
    # 
    # as a workaround I mark all the slashes with additional one, then replace that doubleslashes with # sign
    # then split xpath and unmark them back

    xpath = re.sub(r'//', '#', xpath) # <-- replace doubleslashes with '#'
    pl = xpath.split('/')
    
    xmls = f'<{pl[1]} xmlns="{xmlns}">' if xmlns else f'<{pl[1]}>'
    xmle = f'</{pl[1]}>'
     
    def _xpath2xml(pl):
        key = ''
        xmls = ''
        xmle = ''
        operation_set = ("merge", "replace", "create", "delete", "remove")

        for i in range(len(pl)):
            elem = pl[i]
            if "=" in elem:
                elem,key = elem.split("=", 1)
                key = re.sub(r'#', '/', key) # <-- replace '#'
                xmls += f'<{elem}>{key}</{elem}>'
                break
            if "[]" in elem:
                elem = elem[:-2]
                if operation:
                    if operation not in operation_set:
                        raise ValueError(f'Incorrect operation value\nmust be one of the following: {", ".join(operation_set)}')
                    xmls += f'<{elem} operation="{operation}">'                
                else:
                    xmls += f'<{elem}>'
            else:                    
                xmls += f'<{elem}>'
            xmle = f'</{elem}>' + xmle
     
        if key and i < len(pl)-1:
            return xmls + _xpath2xml(pl[(i+1)::]) + xmle #recursion
        else:
            return xmls + xmle
         
    return xmls + _xpath2xml(pl[2::]) + xmle

def ppxml(xmlstr):
    print(xml.dom.minidom.parseString(xmlstr).toprettyxml(indent="    "))

In [16]:
from scrapli_netconf.driver import NetconfScrape
my_device = {
    "host": "192.168.99.91",
    "auth_username": "apiuser",
    "auth_password": "apipassword",
    "auth_strict_key": False,    
}

conn = NetconfScrape(**my_device)
conn.open()
xmlns = "http://cisco.com/ns/yang/cisco-nx-os-device"

In [17]:
# get all evpn related configs by netconf
vlan_id = 444
vxlan_id = 10444

xpath_nve = f"/System/eps-items/epId-items/Ep-list/epId=1/nws-items/vni-items/Nw-list[]/vni={vxlan_id}"
xpath_svi = f"/System/intf-items/svi-items/If-list[]/id=vlan{vlan_id}"
xpath_vlan = f"/System/bd-items/bd-items/BD-list/fabEncap=vlan-{vlan_id}"
xpath_evpn = f"/System/evpn-items/bdevi-items/BDEvi-list[]/encap=vxlan-{vxlan_id}"

rpc_nve = xpath2xml(xpath_nve, xmlns)
rpc_svi = xpath2xml(xpath_svi, xmlns)
rpc_vlan = xpath2xml(xpath_vlan, xmlns)
rpc_evpn = xpath2xml(xpath_evpn, xmlns)

response = conn.get(filter_=rpc_nve, filter_type="subtree")
print(response.result)
response = conn.get(filter_=rpc_svi, filter_type="subtree")
print(response.result)
response = conn.get(filter_=rpc_vlan, filter_type="subtree")
print(response.result)
response = conn.get(filter_=rpc_evpn, filter_type="subtree")
print(response.result)

<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="101">
  <data>
    <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
      <eps-items>
        <epId-items>
          <Ep-list>
            <epId>1</epId>
            <nws-items>
              <vni-items>
                <Nw-list>
                  <vni>10444</vni>
                  <associateVrfFlag>false</associateVrfFlag>
                  <isLegacyMode>false</isLegacyMode>
                  <mcastGroup>230.1.1.4</mcastGroup>
                  <multisiteIngRepl>disable</multisiteIngRepl>
                  <spineAnyCastGw>false</spineAnyCastGw>
                  <suppressARP>enabled</suppressARP>
                </Nw-list>
              </vni-items>
            </nws-items>
          </Ep-list>
        </epId-items>
      </eps-items>
    </System>
  </data>
</rpc-reply>

<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="102">
  <data>
    <System xmlns="http://cisco.com/ns/yang/cis

In [18]:
# delete evpn
vlan_id = 444
vxlan_id = 10444

del_evpn_conf = ('<config><System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">' + 
        xpath2xml(f"/eps-items/epId-items/Ep-list/epId=1/nws-items/vni-items/Nw-list[]/vni={vxlan_id}", operation="remove") + 
        xpath2xml(f"/intf-items/svi-items/If-list[]/id=vlan{vlan_id}", operation="remove") + 
        xpath2xml(f"/bd-items/bd-items/BD-list[]/fabEncap=vlan-{vlan_id}", operation="remove") + 
        xpath2xml(f"/evpn-items/bdevi-items/BDEvi-list[]/encap=vxlan-{vxlan_id}", operation="remove") + 
"</System></config>")


ppxml(del_evpn_conf)


<?xml version="1.0" ?>
<config>
    <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
        <eps-items>
            <epId-items>
                <Ep-list>
                    <epId>1</epId>
                    <nws-items>
                        <vni-items>
                            <Nw-list operation="remove">
                                <vni>10444</vni>
                            </Nw-list>
                        </vni-items>
                    </nws-items>
                </Ep-list>
            </epId-items>
        </eps-items>
        <intf-items>
            <svi-items>
                <If-list operation="remove">
                    <id>vlan444</id>
                </If-list>
            </svi-items>
        </intf-items>
        <bd-items>
            <bd-items>
                <BD-list operation="remove">
                    <fabEncap>vlan-444</fabEncap>
                </BD-list>
            </bd-items>
        </bd-items>
        <evpn-items>
          

In [19]:
response = conn.edit_config(config=del_evpn_conf, target="running")
print(response.result)

<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="105">
  <ok/>
</rpc-reply>



In [20]:
conn.close()

In [21]:
xmlns = "http://cisco.com/ns/yang/cisco-nx-os-device"
xpath = "/System/intf-items/svi-items/If-list[]/id=vlan111/mtu=9000/descr=blablabla/adminSt=up/rtvrfMbr-items/tDn=//System//inst-items//Inst-list[name='Tenant-1']"

ppxml(xpath2xml(xpath, xmlns=xmlns, operation="replace"))

<?xml version="1.0" ?>
<System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
    <intf-items>
        <svi-items>
            <If-list operation="replace">
                <id>vlan111</id>
                <mtu>9000</mtu>
                <descr>blablabla</descr>
                <adminSt>up</adminSt>
                <rtvrfMbr-items>
                    <tDn>/System/inst-items/Inst-list[name='Tenant-1']</tDn>
                </rtvrfMbr-items>
            </If-list>
        </svi-items>
    </intf-items>
</System>

